In [1]:
import multiprocessing
from multiprocessing.dummy import Pool
from tqdm.auto import tqdm
import imageio
import numpy as np
from glob import glob

In [2]:
n_workers = multiprocessing.cpu_count() // 2

In [3]:
def process_samples(w, s):
    env_name = "SuperMarioBros-%d-%d-v0" % (w + 1, s + 1)
    fnames = glob('./%s/*.mp4' % (env_name))
    fps = imageio.get_reader(fnames[0]).get_meta_data()['fps']
    writer = imageio.get_writer(env_name + '.gif', fps=fps)
    for fname in fnames:
        reader = imageio.get_reader(fname)
        with tqdm(total=len(reader), ascii=True) as pbar:
            for i, im in enumerate(reader):
                writer.append_data(im)
                pbar.update(1)
    writer.close()

In [4]:
with Pool(processes=n_workers) as pool:
    for w in range(8):
        for s in range(4):
            pool.apply_async(process_samples, (w, s))
    pool.close()
    pool.join()